In [1]:
using CPLEXW,Printf, MathProgBase, MathOptInterface, JuMP
using ArgParse

function parse_commandline()
    s = ArgParseSettings()

    @add_arg_table s begin
        "--file", "-f"
            help = "the file name of the model. do not use this option if you have multiple .mps files"
            arg_type = String
            default = 
        "--path", "-p"
            arg_type = String
            default = "."
            help = "the path to the .mps file(s) "
    end

    return parse_args(s)
end

function main()
    parsed_args = parse_commandline()
    println("Parsed args:")
    for (arg,val) in parsed_args
        println("  $arg  =>  $val")
    end
    
    if parsed_args["file"] == nothing
        solve_root_node(true, parsed_args["path"])
    else
        file_name = string(parsed_args["path"],parsed_args["file"]) 
        solve_root_node(false, file_name)
end


function solve_callback(env, cbdata, wherefrom, cbhandle, useraction_p) :: Int32
        println("inside solve_callback")
        nodelp_p = [CPXLPptr(0)]
        CPXgetcallbacknodelp(env, cbdata, wherefrom, nodelp_p);
        CPXwriteprob(env, nodelp_p[1], string(input_file_path,"rootnodemodel.mps"), C_NULL);
        return 0
end 

c_solve_callback = @cfunction(solve_callback, Cint, (
        CPXENVptr,  # env
        Ptr{Cvoid}, # cbdata
        Cint,       # wherefrom
        Ptr{Cvoid}, # cbhandle
        Ptr{Cint},  # useraction_p
    ))

function save_root_cuts(is_batch, path)
    # input file path
    if is_batch == true 
#        input_file_path = "/Users/adminq/Downloads/case1888rte/"
        input_file_path = path
        inputfiles=readdir(input_file_path)
        inputfiles = filter(x->occursin("mps",x), inputfiles)

        for i in eachindex(inputfiles)[1:1]
            status_p = [Cint(0)]
            env = CPXopenCPLEX(status_p)
            CPXsetintparam(env, CPX_PARAM_SCRIND, CPX_ON) # Print to the screen
            CPXsetdblparam(env, CPX_PARAM_NODELIM, 0)      # MIP node limit
            #CPXsetdblparam(env, CPX_PARAM_CUTSFACTOR, 0)  # Disable cutting planes
            #CPXsetintparam(env, CPX_PARAM_PREIND, 0)      # Disable preprocessing
            #CPXsetintparam(env, CPX_PARAM_HEURFREQ, -1)   # Disable primal heuristics
            lp = CPXcreateprob(env, status_p, "problem")
            CPXreadcopyprob(env, lp, string(input_file_path, inputfiles[i]), "mps")
            CPXsetsolvecallbackfunc(env, c_solve_callback, C_NULL)
            CPXmipopt(env, lp);
        end
            collect_root_cuts()
    else
            print("test")
    end

end

CPLEX Error  1013: Bad parameter number to CPLEX parameter routine.
Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  rhs
Selected bound      name:  bounds
Tried aggregator 2 times.
MIP Presolve eliminated 62545 rows and 39794 columns.
MIP Presolve modified 978 coefficients.
Aggregator did 7245 substitutions.
Reduced MIP has 55734 rows, 55874 columns, and 205368 nonzeros.
Reduced MIP has 20782 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.40 sec. (249.53 ticks)
Found incumbent of value 6.8141154e+07 after 0.60 sec. (393.28 ticks)
Probing fixed 132 vars, tightened 6725 bounds.
Probing time = 0.42 sec. (113.70 ticks)
Cover probing fixed 0 vars, tightened 126 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 391 rows and 270 columns.
MIP Presolve modified 13 coefficients.
Reduced MIP has 55343 rows, 55604 columns, and 204112 nonzeros.
Reduced MIP has 20512 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time =

In [4]:
function collect_root_cuts()
    m = JuMP.read_from_file(string(input_file_path,"rootnodemodel.mps"));
        write_to_file(m, string(input_file_path, inputfiles[1], ".lp");format = MOI.FileFormats.FORMAT_LP)
    #cons = all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64});
    cons = [all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64});all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.EqualTo{Float64});all_constraints(m, GenericAffExpr{Float64,VariableRef}, MOI.GreaterThan{Float64})];

    ib_cut_file = open("ib_cuts_analysis.txt", "w"); 
    mir_cut_file = open("mir_cuts_analysis.txt", "w");
    f_cut_file = open("f_cuts_analysis.txt", "w");
    for conRef in cons
        c = constraint_object(conRef);
        varNames = JuMP.name.(keys(c.func.terms))  
        sort!(varNames)
        jointNames = join(varNames)
        csvLine= string(jointNames, ";",name(conRef),";", inputfiles[1], ";",conRef,"\n")       
        println(name(conRef))


        if name(conRef)[begin] == 'i' && isdigit(name(conRef)[2]) # implied bound cuts
            write(ib_cut_file, csvLine)   
        elseif name(conRef)[begin] == 'm' && isdigit(name(conRef)[2]) # MIR cuts
            write(mir_cut_file, csvLine)   
        elseif name(conRef)[begin] == 'f' && isdigit(name(conRef)[2]) # Flow cover cuts
            write(f_cut_file, csvLine)  
        else 
            write(mir_cut_file, csvLine)                                   
            write(f_cut_file, csvLine)
            write(ib_cut_file, csvLine)
        end

    end    

    close(ib_cut_file)     
    close(mir_cut_file)
    close(f_cut_file)
                            
end
                             

c148464
c181316
c158302
c150922
c178418
c156698
c166630
c175847
c172891
c170420
c174160
c175225
c168906
c166788
c169574
c166754
c180273
f3292
c175464
c168454
c164866
i2539
c161071
c170987
c176979
c172732
c176486
c171728
c160361
c171636
c159266
c155884
c160601
c180211
c171794
r7398
c177134
c158454
i2124
m3544
c147112
c154240
f6290
c153745
c179504
c159462
c154468
c156222
c171525
f6353
c163957
i2836
c153690
c162584
i1352
c161487
c164929
c161746
c158689
c160626
c179345
c181225
c170647
f4711
c165684
c167795
c152167
c156202
c168082
c163318
c154921
i1543
c156414
c174252
c161004
c177921
i1353
f3381
c149611
c163481
c154556
c177864
c175083
c173266
c150519
c180332
m4113
i1247
c170854
i640
c157497
c173679
c162055
c174499
i953
c179163
c168757
c159519
c178420
c180564
c161550
c168433
i209
c183346
c183519
i1050
c177731
c158100
c153700
i1584
c182074
f3304
c165731
c146729
c166135
c146863
c167648
c176509
c149136
c162847
c154467
f3496
c179739
c159113
c161903
i2814
L6602
c148939
c166975
c163942
c182859
c15

c174386
c180700
c181309
c164174
c161640
c172672
c165814
c158465
c162426
c174707
c163632
c163296
c146727
c170803
c158113
c176812
c164072
c155685
f6233
c161001
c156650
c169318
c170119
c155794
c162976
c168845
c166705
c180334
c170035
c175005
c152700
c177835
i6667
c158206
c149240
c161240
f6518
c165718
c182517
c160440
i2693
c167115
c183094
c170936
c171076
c176651
c147778
c183107
c166393
f4892
c171953
c154953
c158319
c175200
f4033
c164714
c182978
f4967
c153765
c164238
c167948
c175017
c162229
c158904
c161647
c173819
c149841
c179026
c161703
c166801
c159249
c171990
c151703
c160737
c176489
c156124
f7132
c166084
c169535
c171745
i850
c177981
c159163
c149821
c177049
i939
f7350
c170900
c152194
c170324
c158592
c182786
c174154
c166926
c159837
i2449
c148661
c168067
f6487
c175457
c148907
f5659
f3549
c174303
c165866
i3033
c182725
c168814
c155684
i3044
c169923
c167569
c152823
c165049
c171238
i1738
c156609
c162385
m3669
i417
c161109
c158956
c181079
c165949
c175380
c148243
c161781
c154889
c163416
c165601
c15

c168507
c158207
c172948
f5774
c157027
c153766
c161338
c160431
i6618
c150909
c171351
c182171
c157999
c178199
c161989
c170580
c180488
c178174
c177303
c158119
c175584
m3536
c171904
c167225
c163078
c147945
c171650
c167921
c169101
c170520
c178225
c158922
c162493
c181211
c159538
c152289
c160480
c177291
c163902
c175192
c181447
f5952
c164014
c174792
c168421
c160637
c181886
c159811
i3138
c156579
c159583
c155337
c156463
c175575
c178179
c154938
c173536
i2432
c149121
c156737
c182446
c146325
c158788
c173810
c178670
c160362
c161610
c158087
c148652
c158069
c164861
c167673
i389
c167760
c162116
c174046
c182625
c182378
c182183
c167462
c148510
c168069
c157564
c165428
c157580
c171731
c174540
i6660
c177712
c163386
c163129
c154678
c172290
c173354
f4588
c166817
c164142
c180912
f6436
c147971
c158187
c164536
c169343
c149224
c163989
c164104
i22
f6015
c177037
i226
f4797
f6408
c175266
c167192
f5843
c180801
c172376
c156445
c175505
f5009
c173341
c173329
c156488
c156927
c171596
c177154
c178483
c181201
c165556
c17049

c181411
c157282
c157789
c169237
i3006
c158117
c178599
i2617
i2237
c157974
c175858
f7194
c174724
c156220
f4773
c147798
c155789
c178908
f4492
c162782
c182924
c148932
c179655
c156525
i2774
c175400
c156746
c158000
c177970
c158712
c152705
c178838
f6308
c179379
i1744
c183299
c170444
c165856
c183398
i1504
c166252
c156648
c167529
c170204
i3039
c146489
c175816
m3248
i2092
c176291
c181732
c157533
c169229
c159188
c176384
c171854
c168298
c149598
c173571
c176856
c167503
c165028
c165974
c164659
c156093
c171087
i962
c181002
c152080
c151925
c158086
c173212
c169434
c171208
c148929
c180780
i1092
c178571
f3810
c170335
c152270
c178874
c150073
c166059
c157090
c164835
c174994
c171403
c177130
c160269
c168960
c152678
f6057
i6645
c171002
c172804
f7191
c173224
c173276
f3537
f5925
m3275
c159559
i1835
c177859
c171027
c162898
c169034
c179612
c162497
m3928
c160702
c164701
c156244
c163543
c161705
i872
c180123
c159860
c149504
c164160
c165150
c171546
c151919
c164113
c180537
c161986
c166207
c180719
c170151
c180070
c158

c167311
c182907
c154837
c175536
c165145
c174012
c173227
c175145
c147771
c157362
c153325
i3084
c178057
f7353
c171704
c157597
c152352
c169626
c151373
c162141
c160956
c175669
c176000
c162715
c182434
c160913
c165347
c173523
c153186
c154851
c164095
c165653
c146362
i437
c160226
c166289
c175197
f5698
c149357
c163101
c183606
f6416
c165064
c171820
c168916
c159124
c163147
c181294
c160928
c177519
c170969
c181918
c160164
c161340
c170123
c180165
c173111
i1916
i581
f4662
i2443
c147894
c161680
c180081
c148894
c155248
c182774
c155603
c169056
c178363
c164362
c176726
c155092
c178800
i2506
f5454
c157092
c148249
c181132
c159075
c179534
c156010
c157072
c168972
c165085
c170594
c158301
i1926
c169203
i2239
i1367
c179008
c177690
c180249
m7282
c170073
c153542
i2135
c167338
m7102
c177326
c165288
f5802
i6754
i2982
c167837
f4321
c170608
c172110
i2931
f3565
c158364
f5713
c151350
c160540
c175774
c165916
c177323
c176314
c159346
c149852
c149860
c152206
i1099
c163403
c164435
c162646
c181708
i234
c157869
f5652
c167102
c

...

c122701
c123433
c118722
c123857
c124691
c124737
c124609
c119627
c123457
c119673
c122051
c124104
c123927
c123650
c118658
c118368
c119881
c123011
c119299
c119107
c122383
c120516
c124772
c121792
c121623
c120575
c123611
c122459
c121405
c122641
c118462
c119422
c121505
c120175
c124858
c122455
c124073
c124293
c118317
c121757
c118456
c119270
c121687
c119416
c123195
c120133
c122521
c122545
c122147
c119168
c118743
c122437
c122104
c123523
c120676
c123564
c118577
c118308
c119538
c124666
c122249
c121969
c118314
c120072
c124113
c123990
c122799
c122768
c123487
c120403
c121627
c124395
c120874
c119508
c121406
c121190
c124847
c123304
c124264
c124303
c120730
c123770
c122485
c120491
c124232
c119527
c118338
c123213
c119100
c120986
c118105
c120237
c121253
c123842
c122665
c119692
c119222
c123878
c119636
c121206
c118614
c122832
c119855
c122547
c123000
c118629
c122783
c122920
c120607
c121287
c121132
c119823
c121223
c120028
c123941
c119060
c120922
c118963
c120283
c118190
c122618
c119523
c119131
c122087
c118444


c124733
c118954
c118159
c119577
c121541
c122901
c124180
c121865
c123579
c122465
c122960
c122782
c121759
c124289
c122306
c118552
c120658
c123791
c121748
c123604
c119274
c123301
c124783
c124208
c123780
c119435
c118563
c119596
c122453
c123732
c123996
c119731
c121290
c120694
c120205
c121227
c123428
c121724
c122650
c124230
c123603
c118249
c120449
c118050
c122253
c124892
c124632
c118332
c123377
c122767
c119069
c122115
c124831
c121089
c117642
c118093
c124879
c121632
c123801
c118268
c121213
c124898
c123276
c119393
c118454
c124125
c123918
c121600
c123584
c124366
c119510
c124065
c122555
c118284
c124444
c119531
c118691
c120182
c119484
c122134
c121250
c119832
c121255
c124695
c122430
c119682
c124735
c120745
c118114
c118579
c120419
c124219
c122452
c122424
c121738
c124566
c120530
c120073
c124372
c123015
c120357
c119781
c120729
c119125
c118145
c118437
c122531
c118815
c118811
c120005
c124629
c121360
c119001
c120162
c124201
c122160
c120896
c122741
c119670
c123696
c118302
c118161
c124304
c123666
c118851


c120303
c120879
c119831
c120955
c124237
c120696
c120952
c122077
c123531
c123092
c122883
c121878
c121138
c123884
c118359
c118231
c120838
c118657
c122156
c121074
c118075
c121299
c119509
c120212
c121423
c123744
c122098
c122590
c119901
c123917
c124215
c123270
c124213
c121083
c122427
c121240
c120674
c123999
c123081
c118222
c123290
c121337
c123991
c122692
c124306
c120988
c118188
c124356
c123963
c120451
c120997
c123209
c124687
c120294
c118530
c120259
c120119
c119993
c119754
c119951
c121586
c119549
c123911
c118212
c120253
c124602
c124172
c121021
c122057
c122123
c123694
c119349
c124536
c122570
c122597
c119295
c120152
c120037
c123590
c123722
c119694
c119760
c118907
c119479
c120832
c119716
c123909
c121420
c118570
c120810
c124821
c124608
c119404
c122472
c120317
c121962
c119399
c118387
c123902
c120106
c118676
c122078
c122577
c120396
c122300
c119047
c118923
c120688
c124276
c121746
c124226
c120742
c118345
c123477
c119894
c122151
c122921
c119672
c122669
c122084
c121513
c124204
c119899
c123187
c122675


c123286
c122335
c119689
c120157
c124514
c119588
c119257
c119526
c118887
c123525
c119300
c120769
c122643
c119164
c118595
c124015
c121097
c118776
c120582
c123306
c120691
c121049
c120916
c121212
c122491
c119058
c118219
c118818
c118171
c118147
c122793
c120078
c119605
c118703
c121473
c124605
c118572
c123945
c119015
c123892
c124659
c119530
c124190
c120252
c120868
c122256
c124087
c119887
c118220
c122646
c120134
c122269
c123944
c118700
c124070
c121017
c121808
c124771
c119796
c119952
c122729
c119021
c119105
c120849
c122902
c122105
c121755
c119330
c118899
c119129
c122293
c122461
c119860
c123505
c121798
c118714
c121971
c122974
c119163
c123876
c120404
c122388
c124022
c119160
c120014
c118472
c124561
c123007
c122788
c123754
c122552
c118850
c121618
c123544
c124359
c118376
c120192
c121374
c123497
c124669
c118471
c123895
c124894
c120338
c124703
c123665
c121064
c123870
c120165
c121409
c118285
c122761
c123697
c119858
c118616
c118197
c123353
c121141
c122558
c119808
c123075
c118648
c123456
c123136
c119255


c124808
c119770
c119753
c120889
c120127
c119957
c119856
c124667
c124338
c123051
c120847
c119011
c118297
c118276
c122905
c118242
c123830
c121229
c121462
c119865
c118168
c122677
c119536
c119229
c120500
c118425
c121115
c120339
c119874
c120489
c119381
c119497
c122361
c124248
c122377
c119824
c120567
c123109
c123399
c122386
c118153
c119875
c120767
c122581
c121521
c118470
c118750
c122922
c124450
c124292
c120720
c119544
c118877
c120015
c118868
c120591
c118299
c124010
c122285
c118774
c119615
c121571
c119120
c123271
c119669
c123514
c118953
c118587
c120254
c122374
c119314
c124243
c121325
c119460
c123346
c122563
c123044
c118286
c120101
c123837
c123540
c119589
c122231
c122997
c124549
c118669
c119888
c124837
c122981
c121367
c121186
c120598
c121316
c120122
c120465
c120392
c118813
c122801
c121076
c119367
c123574
c122210
c122088
c122463
c118389
c120707
c118526
c120772
c118248
c123056
c118408
c123245
c121242
c118143
c122712
c121597
c119361
c123729
c124787
c124748
c119077
c121534
c119173
c123541
c119490
